In [1]:
import requests
import os
from datetime import datetime, timedelta
import pandas as pd
import json
import numpy as np


# カレントディレクトリを.pyと合わせるために以下を実行
from pathlib import Path
if Path.cwd().name == "notebook":
    os.chdir("..")


# 設定
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 100)

天気データを結合

In [3]:
# 仙台.csvとdata.csv(平均雲量(10分比)  降雪量合計(cm) 降雪量合計(cm).1  日照時間(時間)等を記載）について、年月日で密結合し、data2.csvに上書き
file = 'data/input/weather_data/我孫子.csv'
df1 = pd.read_csv(file, encoding="shift-jis")
df2 = pd.read_csv('data/input/weather_data/data.csv', encoding="cp932")


df_concat = pd.merge(df1, df2, on='年月日')
print(df_concat[:10])
# 天気概況(昼：06時～18時)を天気(06時～18時)に変更
df_concat = df_concat.rename(columns={'天気概況(昼：06時～18時)': '天気'})
# 天気列について[晴, 曇, 雨, 雪]の文字列のみを抽出する。複数抽出された場合は左側の文字を優先（例：雨曇晴→雨）。みぞれは雨とする。
df_concat['天気'] = df_concat['天気'].str.extract('(晴|曇|雨|雪)', expand=False).fillna('雨')
#df_concat = df_concat.drop(columns=['日照時間(時間).1', '降雪量合計(cm).1','降水量の合計(mm).1','天気概況(夜：18時～翌日06時)'])
df_concat = df_concat.drop(columns=['日照時間(時間).1', '天気概況(夜：18時～翌日06時)'])


df_concat.to_csv('data/input/weather_data/data2.csv', index=False, encoding="cp932")

        年月日  平均気温(℃)  最高気温(℃)  最低気温(℃)  平均風速(m/s)  平均現地気圧(hPa)  平均湿度(％)  \
0       NaN      NaN      NaN      NaN        NaN          NaN      NaN   
1  2020/1/1      3.4      9.0     -1.4        1.3          NaN      NaN   
2  2020/1/2      3.5     10.8     -2.6        1.0          NaN      NaN   
3  2020/1/3      4.1     12.2     -1.7        0.9          NaN      NaN   
4  2020/1/4      4.4     11.1     -1.3        1.4          NaN      NaN   
5  2020/1/5      3.4      8.7     -1.0        2.0          NaN      NaN   
6  2020/1/6      3.8     10.2     -2.9        1.2          NaN      NaN   
7  2020/1/7      3.8      8.2     -0.4        1.3          NaN      NaN   
8  2020/1/8      4.5      6.4      2.7        1.4          NaN      NaN   
9  2020/1/9      9.4     14.0      4.6        3.2          NaN      NaN   

   降水量の合計(mm)  平均雲量(10分比)  降雪量合計(cm) 降雪量合計(cm).1  日照時間(時間) 日照時間(時間).1  \
0         NaN         NaN        NaN      現象なし情報       NaN     現象なし情報   
1         0.0         3.0   

In [4]:
df_concat[:10]

,年月日,平均気温(℃),最高気温(℃),最低気温(℃),平均風速(m/s),平均現地気圧(hPa),平均湿度(％),降水量の合計(mm),平均雲量(10分比),降雪量合計(cm),降雪量合計(cm).1,日照時間(時間),合計全天日射量(MJ/㎡),天気
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,現象なし情報,NaN,NaN,雨
1,2020/1/1,3.4,9.0,-1.4,1.3,NaN,NaN,0.0,3.0,0.0,1,7.4,10.77,晴
2,2020/1/2,3.5,10.8,-2.6,1.0,NaN,NaN,0.0,1.0,0.0,1,5.7,8.28,晴
3,2020/1/3,4.1,12.2,-1.7,0.9,NaN,NaN,0.0,0.0,0.0,1,8.8,11.17,晴
4,2020/1/4,4.4,11.1,-1.3,1.4,NaN,NaN,0.5,5.5,0.0,1,3.4,7.48,曇
5,2020/1/5,3.4,8.7,-1.0,2.0,NaN,NaN,0.5,0.0,0.0,1,8.9,11.76,晴
6,2020/1/6,3.8,10.2,-2.9,1.2,NaN,NaN,0.0,5.8,0.0,1,7.7,11.49,晴
7,2020/1/7,3.8,8.2,-0.4,1.3,NaN,NaN,5.0,9.5,0.0,1,0.5,3.56,曇
8,2020/1/8,4.5,6.4,2.7,1.4,NaN,NaN,7.5,7.5,0.0,1,1.1,3.04,曇
9,2020/1/9,9.4,14.0,4.6,3.2,NaN,NaN,0.0,2.5,0.0,1,8.4,11.89,晴


In [5]:
# data/input/CityToFileMapping.csvを読み込む
city_to_file_mapping = pd.read_csv("data/input/weather_data/CityToFileMapping.csv", encoding="utf-8")

df_weather = pd.DataFrame()
# city_to_file_mappingのfile_nameに.csvがついたファイルをdata/input/から順に読み込み、先頭列にファイル名を付け足し、行方向に結合する
for index, row in city_to_file_mapping.iterrows():
    prefecture = row["県"]
    municipality = row["市"]
    file_name = row["ファイル名"]
    lat = row["lat"]
    lon = row["lon"]
    df_tmp = pd.read_csv(f"data/input/weather_data/{file_name}.csv", encoding="cp932")
    df_tmp['県'] = prefecture
    df_tmp['市'] = municipality
    df_tmp['lat'] = lat
    df_tmp['lon'] = lon
    # df_tmpに「日付」列があれば、それを「年月日」列に変更
    if '日付' in df_tmp.columns:
        df_tmp = df_tmp.rename(columns={'日付': '年月日'})
    
    
    while df_tmp['年月日'].isnull().any():
        df_tmp = df_tmp[1:]
        
    df_weather = pd.concat([df_weather, df_tmp], axis=0)
    # '降水量の合計(mm).1'という列があれば、削除
    if '降水量の合計(mm).1' in df_weather.columns:
        df_weather = df_weather.drop(columns=['降水量の合計(mm).1'])
    
    # 降雪量合計(cm).1という列があれば、削除
    if '降雪量合計(cm).1' in df_weather.columns:
        df_weather = df_weather.drop(columns=['降雪量合計(cm).1'])

        

# df_weatherの列順を年月日、県、市、天気、それ以外の順にする
df_weather = df_weather[['年月日', '県', '市', '天気'] + [col for col in df_weather.columns if col not in ['年月日', '県', '市', '天気']]]

In [9]:
# カスタム関数を定義
def parse_date(date):
    try:
        return pd.to_datetime(date)
    except ValueError:
        try:
            return pd.to_datetime(date, format='%Y年%m月%d日')
        except ValueError:
            return pd.to_datetime(date, format='%Y/%m/%d')

In [23]:
df_weather = df_weather.replace('--', np.nan)
df_weather = df_weather.replace('NaN', np.nan)
df_weather = df_weather.replace('///', np.nan)
df_weather = df_weather.replace('nan', np.nan)


In [11]:
column_types = {
    '県': str,
    '市': str,
    '天気': str,
    '平均気温(℃)': np.float16,
    '最高気温(℃)': np.float16,
    '最低気温(℃)': np.float16,
    '降水量の合計(mm)': np.float16,
    '平均風速(m/s)': np.float16,
    '平均湿度(％)': np.float16,
    '平均現地気圧(hPa)': np.float16,
    'lat': np.double,
    'lon': np.double,
    '平均雲量(10分比)': np.float16,
    '降雪量合計(cm)': np.float16,
    '日照時間(時間)': np.float16,
    '合計全天日射量(MJ/㎡)': np.float16,
}
df_weather['年月日'] = df_weather['年月日'].apply(parse_date)
df_weather['年月日'] = pd.to_datetime(df_weather['年月日'])
df_weather = df_weather.astype(column_types)    # 型変換

In [30]:
df_weather[:10]

,年月日,県,市,天気,平均気温(℃),最高気温(℃),最低気温(℃),降水量の合計(mm),平均風速(m/s),平均湿度(％),平均現地気圧(hPa),lat,lon,平均雲量(10分比),降雪量合計(cm),日照時間(時間),合計全天日射量(MJ/㎡)
0,2020-01-01,福島県,いわき市,NaN,4.699219,9.101562,1.000000,1.5,3.000000,55.0,1020.0,36.990522,140.869525,5.000000,0.0,5.300781,9.132812
1,2020-01-02,福島県,いわき市,NaN,6.101562,10.500000,1.700195,0.0,3.599609,52.0,1020.5,36.990522,140.869525,4.300781,0.0,7.601562,10.226562
2,2020-01-03,福島県,いわき市,NaN,5.101562,8.898438,0.899902,0.0,1.799805,67.0,1016.0,36.990522,140.869525,7.000000,0.0,7.300781,9.671875
3,2020-01-04,福島県,いわき市,NaN,6.101562,10.000000,3.699219,0.0,4.601562,49.0,1014.0,36.990522,140.869525,6.300781,0.0,4.398438,7.160156
4,2020-01-05,福島県,いわき市,NaN,4.199219,7.398438,0.300049,0.0,4.199219,59.0,1020.5,36.990522,140.869525,9.296875,1.0,4.500000,8.062500
5,2020-01-06,福島県,いわき市,NaN,6.699219,10.203125,2.199219,0.0,4.199219,46.0,1025.0,36.990522,140.869525,5.300781,0.0,7.699219,10.812500
6,2020-01-07,福島県,いわき市,NaN,4.898438,8.203125,0.700195,6.5,2.199219,72.0,1025.0,36.990522,140.869525,8.296875,2.0,3.699219,6.921875
7,2020-01-08,福島県,いわき市,NaN,6.101562,8.796875,3.599609,8.0,2.000000,98.0,1003.0,36.990522,140.869525,9.796875,0.0,0.000000,1.900391
8,2020-01-09,福島県,いわき市,NaN,9.601562,13.398438,6.500000,0.0,6.199219,54.0,1008.0,36.990522,140.869525,5.000000,0.0,4.101562,8.078125
9,2020-01-10,福島県,いわき市,NaN,5.800781,9.101562,1.900391,0.0,3.000000,67.0,1016.0,36.990522,140.869525,9.000000,0.0,2.300781,7.488281


不足データは近い地点のもので代替

In [37]:
# 同じ「年月日」と「県」のグループでdf_weather2の文字列NaNをmax()に置き換える
df_weather['天気'] = df_weather.groupby(['年月日', '県'])['天気'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['平均気温(℃)'] = df_weather.groupby(['年月日', '県'])['平均気温(℃)'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['最高気温(℃)'] = df_weather.groupby(['年月日', '県'])['最高気温(℃)'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['最低気温(℃)'] = df_weather.groupby(['年月日', '県'])['最低気温(℃)'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['降水量の合計(mm)'] = df_weather.groupby(['年月日', '県'])['降水量の合計(mm)'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['日照時間(時間)'] = df_weather.groupby(['年月日', '県'])['日照時間(時間)'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['降雪量合計(cm)'] = df_weather.groupby(['年月日', '県'])['降雪量合計(cm)'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['平均風速(m/s)'] = df_weather.groupby(['年月日', '県'])['平均風速(m/s)'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['平均湿度(％)'] = df_weather.groupby(['年月日', '県'])['平均湿度(％)'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['平均雲量(10分比)'] = df_weather.groupby(['年月日', '県'])['平均雲量(10分比)'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['合計全天日射量(MJ/㎡)'] = df_weather.groupby(['年月日', '県'])['合計全天日射量(MJ/㎡)'].transform(lambda x: x.fillna(x.dropna().max()))
df_weather['平均現地気圧(hPa)'] = df_weather.groupby(['年月日', '県'])['平均現地気圧(hPa)'].transform(lambda x: x.fillna(x.dropna().max()))


In [58]:
df_weather.to_csv('data/input/weather_data/weather.csv', index=False, encoding="utf-8")

point_historyと結合

In [78]:
df_point_history = pd.read_csv('data/input/point_history_cleansing_2.csv', encoding="utf-8")  # point_history_2.csv（都道府県、市を含む）を読み込む
df_point_history[:3]

C:\Users\monch\AppData\Local\Temp\ipykernel_27796\1625372275.py:1: DtypeWarning: Columns (5,6,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_point_history = pd.read_csv('data/input/point_history_cleansing_2.csv', encoding="utf-8")  # point_history_2.csv（都道府県、市を含む）を読み込む


,Unnamed: 0,id,user_id,series_id,shop_id,shop_name,card_id,リサイクル分類ID,amount,amount_kg,point,total_point,status,total_amount,coin,rank_id,use_date,created_at,updated_at,支店ID,super,prefectures,municipality,shop_name_1,shop_id_1,created_at_1,updated_at_1,store_opening_time,store_closing_time,rps_opening_time,rps_closing_time,store_latitude,store_longitude
0,0,22,2023,1.0,52.0,サン・マルシェ東中田店,superbig047780,1.0,2.0,2.0,2.0,98.0,1,NaN,0.0,0.0,2020-12-17 18:01:24,2020-12-17 18:01:24,2021-04-14 17:57:46,19.0,サン・マルシェ,宮城県,仙台市太白区,東中田店,52.0,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.202796,140.901174
1,1,27,90,1.0,52.0,サン・マルシェ東中田店,superbig011187,1.0,7.0,7.0,7.0,143.0,1,NaN,21.0,2.0,2020-12-18 10:59:56,2020-12-18 10:59:56,2020-12-31 13:57:54,19.0,サン・マルシェ,宮城県,仙台市太白区,東中田店,52.0,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.202796,140.901174
2,2,34,670,1.0,52.0,サン・マルシェ東中田店,superbig035909,1.0,5.0,5.0,5.0,7.0,1,NaN,5.0,0.0,2020-12-18 14:30:36,2020-12-18 14:30:36,2020-12-31 13:58:58,19.0,サン・マルシェ,宮城県,仙台市太白区,東中田店,52.0,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.202796,140.901174


In [79]:
# use_date列をparse_date関数で日付型に変換し、時間は切り捨てし、[use_date_2]列に格納
df_point_history['use_date_2'] = pd.to_datetime(df_point_history['use_date']).dt.floor('d')
df_point_history[:3]

,Unnamed: 0,id,user_id,series_id,shop_id,shop_name,card_id,リサイクル分類ID,amount,amount_kg,point,total_point,status,total_amount,coin,rank_id,use_date,created_at,updated_at,支店ID,super,prefectures,municipality,shop_name_1,shop_id_1,created_at_1,updated_at_1,store_opening_time,store_closing_time,rps_opening_time,rps_closing_time,store_latitude,store_longitude,use_date_2
0,0,22,2023,1.0,52.0,サン・マルシェ東中田店,superbig047780,1.0,2.0,2.0,2.0,98.0,1,NaN,0.0,0.0,2020-12-17 18:01:24,2020-12-17 18:01:24,2021-04-14 17:57:46,19.0,サン・マルシェ,宮城県,仙台市太白区,東中田店,52.0,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.202796,140.901174,2020-12-17
1,1,27,90,1.0,52.0,サン・マルシェ東中田店,superbig011187,1.0,7.0,7.0,7.0,143.0,1,NaN,21.0,2.0,2020-12-18 10:59:56,2020-12-18 10:59:56,2020-12-31 13:57:54,19.0,サン・マルシェ,宮城県,仙台市太白区,東中田店,52.0,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.202796,140.901174,2020-12-18
2,2,34,670,1.0,52.0,サン・マルシェ東中田店,superbig035909,1.0,5.0,5.0,5.0,7.0,1,NaN,5.0,0.0,2020-12-18 14:30:36,2020-12-18 14:30:36,2020-12-31 13:58:58,19.0,サン・マルシェ,宮城県,仙台市太白区,東中田店,52.0,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.202796,140.901174,2020-12-18


In [80]:
# '市'または'群'に続く文字を削除
df_point_history['municipality'] = df_point_history['municipality'].str.replace(r'(市|郡).*', r'\1', regex=True)

In [81]:
df_point_history.loc[df_point_history['municipality'] == "群"].loc['municipality'] = "郡山群"
df_point_history.loc[df_point_history['municipality'] == "塩竉市"].loc['municipality'] = "塩竈市"

C:\Users\monch\AppData\Local\Temp\ipykernel_27796\42921894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_point_history.loc[df_point_history['municipality'] == "群"].loc['municipality'] = "郡山群"
C:\Users\monch\AppData\Local\Temp\ipykernel_27796\42921894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_point_history.loc[df_point_history['municipality'] == "塩竉市"].loc['municipality'] = "塩竈市"


In [82]:
unique_municipalities = df_point_history['municipality'].unique()
unique_municipalities

array(['仙台市', nan, '柴田郡', '白石市', '角田市', 'にかほ市', '秋田市', '大仙市', '新庄市',
       '花巻市', '横手市', '湯沢市', '富谷市', '宇都宮市', '西白河郡', '東白川郡', '須賀川市', '石巻市',
       '福島市', '登米市', '大田原市', '多賀城市', '本宮市', '水戸市', '郡', '米沢市', '長井市',
       '会津若松市', '南相馬市', '石川郡', '牛久市', '伊達市', '宮城郡', '東根市', '田村郡', '田村市',
       '耶麻郡', '名取市', '相馬市', 'いわき市', '取手市', '栃木市', '茨城市', '喜多方市', '東松島市',
       '大崎市', '岩沼市', '栗原市', '亘理郡', 'ひたちなか市', '下野市', '東茨城郡', '佐野市', '黒川郡',
       '石岡市', 'つくばみらい市', '矢板市', 'かすみがうら市', '那須塩原市', '小山市', '天童市', '稲敷郡',
       '日立市', '鹿沼市', 'つくば市', '土浦市', 'さくら市', '結城市', '山形市', '小美玉市', '遠田郡',
       '寒河江市', '那珂郡', '足利市', '守谷市', '塩釜市', '筑西市', '笠間市', '加美郡', '北茨城市',
       '南陽市', '東置賜郡', '西村山郡', '上山市', '坂東市', '二本松市', '白河市', '南会津郡', '福水戸市',
       '常陸大宮市', '塩竈市', '伊達郡', '河沼郡', '塩竉市', '常陸太田市', '野田市', '新潟市', '村上市',
       'さいたま市', '柏市'], dtype=object)

In [83]:
# 'use_date_2'列をdatetime型に変換
df_point_history['use_date_2'] = pd.to_datetime(df_point_history['use_date_2'])

# '年月日'列をdatetime型に変換
df_weather['年月日'] = pd.to_datetime(df_weather['年月日'])

# df_point_historyのmunicipalityとuse_date_2、df_weatherの市と年月日で結合し、df_point_history_weatherに格納
df_point_history_weather = pd.merge(df_point_history, df_weather, left_on=['municipality', 'use_date_2'], right_on=['市', '年月日'], how='left')
df_point_history_weather[:3]

,Unnamed: 0,id,user_id,series_id,shop_id,shop_name,card_id,リサイクル分類ID,amount,amount_kg,point,total_point,status,total_amount,coin,rank_id,use_date,created_at,updated_at,支店ID,super,prefectures,municipality,shop_name_1,shop_id_1,created_at_1,updated_at_1,store_opening_time,store_closing_time,rps_opening_time,rps_closing_time,store_latitude,store_longitude,use_date_2,年月日,県,市,天気,平均気温(℃),最高気温(℃),最低気温(℃),降水量の合計(mm),平均風速(m/s),平均湿度(％),平均現地気圧(hPa),lat,lon,平均雲量(10分比),降雪量合計(cm),日照時間(時間),合計全天日射量(MJ/㎡)
0,0,22,2023,1.0,52.0,サン・マルシェ東中田店,superbig047780,1.0,2.0,2.0,2.0,98.0,1,NaN,0.0,0.0,2020-12-17 18:01:24,2020-12-17 18:01:24,2021-04-14 17:57:46,19.0,サン・マルシェ,宮城県,仙台市,東中田店,52.0,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.202796,140.901174,2020-12-17,2020-12-17,宮城県,仙台市,雪,-0.700195,1.200195,-2.300781,7.5,2.599609,85.0,1009.0,38.26931,140.881704,8.296875,9.0,1.5,4.878906
1,1,27,90,1.0,52.0,サン・マルシェ東中田店,superbig011187,1.0,7.0,7.0,7.0,143.0,1,NaN,21.0,2.0,2020-12-18 10:59:56,2020-12-18 10:59:56,2020-12-31 13:57:54,19.0,サン・マルシェ,宮城県,仙台市,東中田店,52.0,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.202796,140.901174,2020-12-18,2020-12-18,宮城県,仙台市,晴,1.599609,4.800781,-2.400391,0.0,2.000000,81.0,1010.0,38.26931,140.881704,3.300781,0.0,3.0,5.398438
2,2,34,670,1.0,52.0,サン・マルシェ東中田店,superbig035909,1.0,5.0,5.0,5.0,7.0,1,NaN,5.0,0.0,2020-12-18 14:30:36,2020-12-18 14:30:36,2020-12-31 13:58:58,19.0,サン・マルシェ,宮城県,仙台市,東中田店,52.0,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.202796,140.901174,2020-12-18,2020-12-18,宮城県,仙台市,晴,1.599609,4.800781,-2.400391,0.0,2.000000,81.0,1010.0,38.26931,140.881704,3.300781,0.0,3.0,5.398438


In [88]:
df_point_history_weather.to_csv('data/input/point_history_cleansing_weather.csv', index=False, encoding="utf-8")

試しに分析

In [95]:
df = pd.read_csv('data/input/point_history_cleansing_weather.csv', encoding="utf-8")

C:\Users\monch\AppData\Local\Temp\ipykernel_27796\1241416526.py:1: DtypeWarning: Columns (5,6,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/input/point_history_cleansing_weather.csv', encoding="utf-8")


In [96]:
# 天気でグループ化し、amount_kgの平均を取得
df.groupby('天気')['amount_kg'].mean()

天気
晴    9.197395
曇    9.120061
雨    8.652991
雪    8.541589
Name: amount_kg, dtype: float64

# 以降はボツ

In [2]:


def get_weather_forecast(api_key, lat, lon):
    """
    緯度経度を指定してOpenWeatherMap APIから天気予報を取得する
    arges:
        api_key: OpenWeatherMap APIのキー
        lat: 緯度
        lon: 経度
    return:
        天気データのリスト
    """
    url = "https://api.openweathermap.org/data/2.5/forecast"
    
    weather_data = []
    params = {
            'lat': lat,
            'lon': lon,
            'appid': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        weather_data.extend(data.get('list', []))
    else:
        print(response)
    
    return weather_data




In [3]:
# get_weather_from_cityから仙台市の天気予報を取得
lat = 38.2682
lon = 140.8694
api_key = os.environ["WEATHER_API_KEY"]

jsondata = get_weather_forecast(api_key, lat, lon)
jsondata

[{'dt': 1702890000,
  'main': {'temp': 274.79,
   'feels_like': 269.1,
   'temp_min': 273.69,
   'temp_max': 274.79,
   'pressure': 1024,
   'sea_level': 1024,
   'grnd_level': 1020,
   'humidity': 55,
   'temp_kf': 1.1},
  'weather': [{'id': 802,
    'main': 'Clouds',
    'description': 'scattered clouds',
    'icon': '03n'}],
  'clouds': {'all': 40},
  'wind': {'speed': 7.62, 'deg': 295, 'gust': 17.17},
  'visibility': 10000,
  'pop': 0.25,
  'sys': {'pod': 'n'},
  'dt_txt': '2023-12-18 09:00:00'},
 {'dt': 1702900800,
  'main': {'temp': 274.44,
   'feels_like': 270.01,
   'temp_min': 273.75,
   'temp_max': 274.44,
   'pressure': 1025,
   'sea_level': 1025,
   'grnd_level': 1021,
   'humidity': 57,
   'temp_kf': 0.69},
  'weather': [{'id': 802,
    'main': 'Clouds',
    'description': 'scattered clouds',
    'icon': '03n'}],
  'clouds': {'all': 33},
  'wind': {'speed': 4.71, 'deg': 292, 'gust': 14.15},
  'visibility': 10000,
  'pop': 0.21,
  'sys': {'pod': 'n'},
  'dt_txt': '2023-12-1

In [19]:
print("天気：",jsondata[0]["weather"][0]["main"])
print("天気詳細：",jsondata[0]["weather"][0]["description"])

print("気温：",jsondata[0]["main"]["temp"])
print("最高気温：",jsondata[0]["main"]["temp_max"])
print("最低気温：",jsondata[0]["main"]["temp_min"])
print("体感気温：",jsondata[0]["main"]["feels_like"])
print("最低気温：",jsondata[0]["main"]["temp_min"])
print("最高気温：",jsondata[0]["main"]["temp_max"])
print("気圧：",jsondata[0]["main"]["grnd_level"])
print("湿度：",jsondata[0]["main"]["humidity"])
#print("降水量：",jsondata[0]["rain"]["3h"])

print("風速：",jsondata[0]["wind"]["speed"])
print("風の方角：",jsondata[0]["wind"]["deg"])
print("雲量：",jsondata[0]["clouds"]["all"])
print("降水確率：",jsondata[0]["pop"])
print("日時：",jsondata[0]["dt_txt"])

天気： Clouds
天気詳細： scattered clouds
気温： 274.79
最高気温： 274.79
最低気温： 273.69
体感気温： 269.1
最低気温： 273.69
最高気温： 274.79
気圧： 1020
湿度： 55
風速： 7.62
風の方角： 295
雲量： 40
降水確率： 0.25
日時： 2023-12-18 09:00:00


In [17]:
#jsondataを保存
with open('data/weatherdata.json', 'w') as f:
    json.dump(jsondata, f, indent=4)

In [4]:
def get_weather_history(api_key, lat, lon):
    """
    緯度経度から天気データを取得する
    arges:
        api_key: OpenWeatherMapのAPIキー
        lat: 緯度
        lon: 経度
    return:
        天気データのリスト

    """
    url = "https://api.openweathermap.org/data/2.5/onecall/timemachine"
    
    weather_data = []
    params = {
            'lat': lat,
            'lon': lon,
            'appid': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        weather_data.extend(data.get('list', []))
    else:
        print(response)
        print(f"Error fetching data for {start_date.strftime('%Y-%m-%d')}: {response.status_code}")
    
    return weather_data

In [27]:
df = pd.read_csv('data/input/point_history_cleansing_2.csv')

C:\Users\monch\AppData\Local\Temp\ipykernel_15860\823842268.py:1: DtypeWarning: Columns (5,6,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/input/point_history_cleansing_2.csv')


In [28]:
# dfのuse_dateがに並び替え
df = df.sort_values('use_date')
df[:10]

,Unnamed: 0,id,user_id,series_id,shop_id,shop_name,card_id,リサイクル分類ID,amount,amount_kg,point,total_point,status,total_amount,coin,rank_id,use_date,created_at,updated_at,支店ID,super,prefectures,municipality,shop_name_1,shop_id_1,created_at_1,updated_at_1,store_opening_time,store_closing_time,rps_opening_time,rps_closing_time,store_latitude,store_longitude
8704,9151,1037309,912,3.0,168.0,ビフレ新庄店,315bifre056848,1.0,0.0,0.0,1.0,77.0,1,NaN,0.0,0.0,2020-01-01 10:48:51,2020-11-05 23:07:54,2020-12-05 10:06:56,7.0,ビフレ,山形県,新庄市,新庄店,168.0,2021-09-21 04:12:05,2023-04-07 16:24:30,09:00:00,20:50:00,09:00:00,20:00:00,38.760142,140.288781
8705,9152,1037329,1785,3.0,168.0,ビフレ新庄店,315bifre012638,1.0,0.0,0.0,1.0,135.0,1,NaN,0.0,0.0,2020-01-01 16:19:14,2020-11-05 23:07:55,2021-02-12 12:30:51,7.0,ビフレ,山形県,新庄市,新庄店,168.0,2021-09-21 04:12:05,2023-04-07 16:24:30,09:00:00,20:50:00,09:00:00,20:00:00,38.760142,140.288781
8706,9153,1037361,912,3.0,168.0,ビフレ新庄店,315bifre056848,1.0,0.0,0.0,1.0,77.0,1,NaN,0.0,0.0,2020-01-02 10:35:10,2020-11-05 23:12:42,2020-12-05 10:06:56,7.0,ビフレ,山形県,新庄市,新庄店,168.0,2021-09-21 04:12:05,2023-04-07 16:24:30,09:00:00,20:50:00,09:00:00,20:00:00,38.760142,140.288781
8707,9154,1037366,1785,3.0,168.0,ビフレ新庄店,315bifre012638,1.0,0.0,0.0,1.0,135.0,1,NaN,0.0,0.0,2020-01-02 11:44:31,2020-11-05 23:12:42,2021-02-12 12:30:51,7.0,ビフレ,山形県,新庄市,新庄店,168.0,2021-09-21 04:12:05,2023-04-07 16:24:30,09:00:00,20:50:00,09:00:00,20:00:00,38.760142,140.288781
8708,9155,1037385,1955,3.0,168.0,ビフレ新庄店,315bifre060288,1.0,0.0,0.0,1.0,149.0,1,NaN,0.0,0.0,2020-01-02 15:19:16,2020-11-05 23:12:42,2021-04-03 13:09:13,7.0,ビフレ,山形県,新庄市,新庄店,168.0,2021-09-21 04:12:05,2023-04-07 16:24:30,09:00:00,20:50:00,09:00:00,20:00:00,38.760142,140.288781
8709,9156,1037390,1017,3.0,168.0,ビフレ新庄店,315bifre051699,1.0,0.0,0.0,1.0,137.0,1,NaN,0.0,0.0,2020-01-02 15:49:49,2020-11-05 23:12:43,2020-11-24 12:20:41,7.0,ビフレ,山形県,新庄市,新庄店,168.0,2021-09-21 04:12:05,2023-04-07 16:24:30,09:00:00,20:50:00,09:00:00,20:00:00,38.760142,140.288781
8710,9157,1037391,2605,3.0,168.0,ビフレ新庄店,315bifre056464,1.0,0.0,0.0,1.0,16.0,1,NaN,0.0,0.0,2020-01-02 15:58:44,2020-11-05 23:12:43,2021-08-27 09:19:21,7.0,ビフレ,山形県,新庄市,新庄店,168.0,2021-09-21 04:12:05,2023-04-07 16:24:30,09:00:00,20:50:00,09:00:00,20:00:00,38.760142,140.288781
8711,9158,1037411,1108,3.0,205.0,ビフレ御野場店,315bifre064796,1.0,0.0,0.0,1.0,77.0,1,NaN,0.0,0.0,2020-01-02 16:44:43,2020-11-05 23:12:43,2021-01-07 10:56:27,11.0,ビフレ,秋田県,秋田市,御野場店,205.0,2021-09-21 04:16:07,2023-04-08 15:41:54,09:30:00,21:00:00,09:30:00,19:00:00,39.672817,140.136120
8712,9159,1037414,2922,3.0,205.0,ビフレ御野場店,315bifre059902,1.0,0.0,0.0,1.0,171.0,1,NaN,0.0,0.0,2020-01-02 17:14:42,2020-11-05 23:12:43,2021-09-21 09:45:00,11.0,ビフレ,秋田県,秋田市,御野場店,205.0,2021-09-21 04:16:07,2023-04-08 15:41:54,09:30:00,21:00:00,09:30:00,19:00:00,39.672817,140.136120
8713,9160,1037415,1102,3.0,205.0,ビフレ御野場店,315bifre059955,1.0,0.0,0.0,1.0,70.0,1,NaN,0.0,0.0,2020-01-02 18:04:39,2020-11-05 23:12:44,2020-11-29 09:19:42,11.0,ビフレ,秋田県,秋田市,御野場店,205.0,2021-09-21 04:16:07,2023-04-08 15:41:54,09:30:00,21:00:00,09:30:00,19:00:00,39.672817,140.136120


In [29]:
# '市'または'群'に続く文字を削除
df['municipality'] = df['municipality'].str.replace(r'(市|郡).*', r'\1', regex=True)
unique_municipalities = df['municipality'].unique()
unique_municipalities

array(['新庄市', '秋田市', '仙台市', nan, '柴田郡', 'にかほ市', '湯沢市', '横手市', '大仙市',
       '花巻市', '白石市', '角田市', '福島市', '石巻市', '黒川郡', '富谷市', '宮城郡', '塩竈市',
       '宇都宮市', '西白河郡', '東白川郡', '須賀川市', '登米市', '大田原市', '多賀城市', '本宮市',
       '水戸市', '郡', '米沢市', '長井市', '会津若松市', '南相馬市', '石川郡', '牛久市', '伊達市',
       '東根市', '田村郡', '田村市', '耶麻郡', '名取市', '相馬市', 'いわき市', '取手市', '栃木市',
       '茨城市', '喜多方市', '東松島市', '大崎市', '岩沼市', '栗原市', '亘理郡', 'ひたちなか市', '下野市',
       '東茨城郡', '佐野市', '石岡市', 'つくばみらい市', '矢板市', 'かすみがうら市', '那須塩原市', '小山市',
       '天童市', '稲敷郡', '日立市', '鹿沼市', '上山市', 'つくば市', '土浦市', 'さくら市', '結城市',
       '山形市', '小美玉市', '遠田郡', '二本松市', '南陽市', '寒河江市', '北茨城市', '那珂郡', '足利市',
       '東置賜郡', '白河市', '加美郡', '守谷市', '塩釜市', '筑西市', '笠間市', '坂東市', '西村山郡',
       '南会津郡', '福水戸市', '常陸大宮市', '伊達郡', '河沼郡', '塩竉市', '常陸太田市', '野田市',
       '村上市', '新潟市', 'さいたま市', '柏市'], dtype=object)

In [37]:
df_municipality = df.groupby(['prefectures', 'municipality'])

In [30]:
# 各市町村のstore_latitude	store_longitudeの平均を求める
date_range_per_municipality = df.groupby(['prefectures', 'municipality']).agg({'store_latitude': 'mean', 'store_longitude': 'mean'})
date_range_per_municipality

store_latitude  store_longitude
prefectures municipality                                 
千葉県         柏市                 35.879605       139.967660
            野田市                35.981155       139.853231
埼玉県         さいたま市              35.852908       139.627677
宮城県         亘理郡                38.037486       140.863861
            仙台市                38.269310       140.881704
            加美郡                38.577096       140.852431
            名取市                38.172068       140.877090
            塩竈市                38.322669       141.003351
            塩竉市                38.311792       141.032019
            塩釜市                38.322136       141.023963
            多賀城市               38.289480       140.993456
            大崎市                38.572614       140.960612
            宮城郡                38.317751       140.976434
            富谷市                38.350625       140.891217
            岩沼市                38.109975       140.865342
            東松島市               38.423260       141.215057
            柴田郡                38.058843       140.745633
            栗原市                38.754769       141.076413
            登米市                38.697238       141.206527
            白石市                37.999404       140.614751
            石巻市                38.433972       141.294348
            角田市                37.979140       140.784359
            遠田郡                38.541008       141.082572
            黒川郡                38.418594       140.883845
山形県         上山市                38.152381       140.288284
            南陽市                38.053120       140.153152
            天童市                38.354870       140.376595
            寒河江市               38.381667       140.260556
            山形市                38.253358       140.321089
            新庄市                38.760220       140.290550
            東根市                38.417733       140.385465
            東置賜郡               38.000168       140.185580
            米沢市                37.916726       140.110546
            西村山郡               38.430552       140.320881
            長井市                38.090068       140.037816
岩手県         花巻市                39.372329       141.119064
新潟県         新潟市                37.913363       139.080788
            村上市                38.225296       139.468945
栃木県         さくら市               36.692512       139.964282
            下野市                36.468594       139.868189
            佐野市                36.306624       139.583195
            大田原市               36.868590       140.017938
            宇都宮市               36.546848       139.884672
            小山市                36.304157       139.811273
            栃木市                36.374685       139.728067
            矢板市                36.798426       139.929132
            足利市                36.323697       139.458019
            那須塩原市              36.923974       140.007581
            鹿沼市                36.561944       139.764942
福島県         いわき市               36.990522       140.869525
            二本松市               37.590948       140.439703
            伊達市                37.820104       140.539826
            伊達郡                37.874277       140.541428
            会津若松市              37.492906       139.925711
            南会津郡               37.203173       139.769949
            南相馬市               37.632717       140.963737
            喜多方市               37.649904       139.881566
            本宮市                37.502453       140.389980
            東白川郡               37.037270       140.387858
            河沼郡                37.566993       139.807244
            田村市                37.447284       140.581988
            田村郡                37.392004       140.534222
            白河市                37.126236       140.203979
            相馬市                37.804609       140.913747
            石川郡                37.155040       140.446241
            福島市                37.769872       140.460131
            耶麻郡                37.557258       140.111198
            西白河郡        